In [1]:
import pandas as pd
import numpy as np
import json
import random

In [2]:
n_features = 5000
n_components = 5
n_top_words = 15

In [3]:
#lda_topics = ['talk.politics.mideast', 'rec.sport.hockey', 'soc.religion.christian', 'sci.crypt', 'comp.graphics']

In [4]:
labels = ["Direct Harassment", "Hate Speech","Sexual Harassment","Trolling", "Others", "Toxic"]

batch_files = ['usanews.csv', 'foxnews0.csv', 'foxnews1.csv', 'foxnews2.csv', 'foxnews3.csv', 
                'nogla0.csv','pew0.csv', 'rae0_0.csv', 
                 'rae0_1.csv', 'terror0_0.csv', 'terror0_1.csv']

vid_files = ["foxnews_FULL.csv", "rae_FULL.csv",  "terror_FULL.csv",
             "usanews.csv" , "pew0.csv", "nogla0.csv", "drdisrespect_FULL.csv"]
genre_files = ["gaming_channels_GENRE.csv", "news_channel_GENRE.csv" ]

files = []
files.extend(vid_files)
files.extend(genre_files)
print(files)

['foxnews_FULL.csv', 'rae_FULL.csv', 'terror_FULL.csv', 'usanews.csv', 'pew0.csv', 'nogla0.csv', 'drdisrespect_FULL.csv', 'gaming_channels_GENRE.csv', 'news_channel_GENRE.csv']


In [93]:
from sklearn.feature_extraction.text import CountVectorizer
def get_tf(doc):
    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features, stop_words='english', ngram_range=(1, 1))
    vector = tf_vectorizer.fit_transform(doc)
    feature_names = np.array(tf_vectorizer.get_feature_names())
    return pd.DataFrame(vector.todense(), columns=feature_names), vector, feature_names
    

In [101]:
src_folder = "../Annotations/CSVs/"
df = pd.read_csv(src_folder + "news_channel_GENRE.csv", index_col = 0)
df["words"] = df["words"].replace(np.nan, '', regex=True).apply(str)
tf_df, vect, features = get_tf(df["words"] )

In [102]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
def get_keywords(doc):
    nmf = NMF(n_components=2,random_state=45)
    vector = nmf.fit_transform(doc)
    return nmf, vector

In [103]:
nmf, vector = get_keywords(vect)

In [104]:
for index, topic in enumerate(nmf.components_):
    print(f'Top 15 words for Topic #{index}')
    print([features[i] for i in topic.argsort()[:5]])
    print('\n')

Top 15 words for Topic #0
['00', 'people', 'peoples', 'peopleâ', 'pepper']


Top 15 words for Topic #1
['𝗨𝗦', 'ûœ', 'ùˆù', 'ùšù', 'mðÿ']




In [98]:
src_folder = "../Annotations/CSVs/"
df = pd.read_csv(src_folder + "gaming_channels_GENRE.csv", index_col = 0)
df["words"] = df["words"].replace(np.nan, '', regex=True).apply(str)
tf_df, vect, features = get_tf(df[df['Toxic'] == 1]["cleaned_message"])

C:\Users\Phoebe\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [99]:
nmf, vector = get_keywords(vect)

In [100]:
for index, topic in enumerate(nmf.components_):
    print(f'Top 15 words for Topic #{index}')
    print([features[i] for i in topic.argsort()[:10]])
    print('\n')

Top 15 words for Topic #0
['bitch', 'blm', 'lives', 'shut', 'chat', 'death', 'dick', 'fuckers', 'floyd', 'pete']


Top 15 words for Topic #1
['white', 'trash', 'boys', 'shit', 'pete', 'like', 'vote', 'fucking', 'fuckin', 'racist']


